# Scraping Data for over 2000 Movies :
Selon le site : https://www.dataquest.io/blog/web-scraping-beautifulsoup/

It's essential to identify the goal of our scraping right from the beginning. Writing a scraping script can take a lot of time, especially if we want to scrape more than one web page. We want to avoid spending hours writing a script which scrapes data we won't actually need.

## Il est très important de savoir pourquoi on Scrape à la base

## Il faut faire attention au temps de calcul

In [1]:
import requests

url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = requests.get(url)
response

<Response [200]>

In [2]:
print(response.text[:500])





<!DOCTYPE html>
<html
xmlns:og="http://ogp.me/ns#"
xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 


### Utiliser BeautifulSoup pour le parsing

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

##### Extraire tous les elts contenant les films et leurs infos

In [4]:
movie_containers = soup.find_all('div',class_='lister-item mode-advanced')# renvoie un objet ResultSet sous forme de liste
print(len(movie_containers))
type(movie_containers)

50


bs4.element.ResultSet

### Extraire les données suivantes depuis les conteneurs : 

Now we'll select only the first container, and extract, by turn, each item of interest:

  * The name of the movie.
  * The year of release.
  * The IMDB rating.
  * The Metascore.
  * The number of votes.


### Extraire les Données pour un seul Film

In [5]:
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/?ref_=adv_li_i"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://ia.media-imdb.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB499613450_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/?ref_=adv_li_tt">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted ">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</s

##### Nom du film :         Name

In [6]:
first_movie.h3.a.text # plus aisé à trouver vu qu'il n'y a pas plusieurs éléments de concurrents de même type

'Logan'

In [7]:
first_name = first_movie.h3.a.text 

##### Année du film :     Year

In [8]:
first_year = first_movie.find('span',class_='lister-item-year').text
first_year

'(2017)'

##### Note IMDB :       IMDB Rating

In [9]:
first_imdb = float(first_movie.strong.text)
first_imdb

8.1

##### Note Metascore :        Metascore

In [10]:
first_metascore = int(first_movie.find('span',class_='metascore').text)
first_metascore

77

##### Nombre de votant : votes

In [11]:
first_votes = first_movie.find('span',attrs={'name':'nv'})
print(first_votes)

<span data-value="483078" name="nv">483,078</span>


In [12]:
votes_int = int(first_votes['data-value'])

In [13]:
votes_int

483078

### Script pour extraire les films de toutes une page

Certains films n ont pas de note metascore, il ne faudra donc pas les prendre en compte lors de l extraction

In [14]:
""" Liste des listes de données"""
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

In [15]:
for container in movie_containers:
    if container.find('div',class_='ratings-metascore') is not None:
        
        #noms
        names.append(container.h3.a.text)
        #annees
        years.append(container.find('span',class_='lister-item-year').text)
        #imdb
        imdb_ratings.append(float(container.strong.text))
        #metascore
        metascores.append(int(container.find('span',class_='metascore').text))
        #votes
        v = container.find('span',attrs={'name':'nv'})
        votes.append(int(v['data-value']) )

In [16]:
votes

[483078,
 412999,
 390888,
 383070,
 381605,
 335389,
 326916,
 306545,
 300342,
 296172,
 275289,
 253411,
 237440,
 229503,
 215878,
 207629,
 206111,
 199729,
 195619,
 180283,
 175771,
 171718,
 165174,
 161049,
 157601,
 152908,
 148473,
 137731,
 132999,
 131703,
 126833,
 125821,
 124557,
 123362,
 123084,
 118136,
 117071,
 107009,
 104810,
 99744,
 98680,
 98559,
 97990,
 90325,
 90048,
 85744]

### Introduction Pandas

In [17]:
import pandas as pd

df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 5 columns):
imdb         46 non-null float64
metascore    46 non-null int64
movie        46 non-null object
votes        46 non-null int64
year         46 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ KB


In [19]:
df.head(10)

,imdb,metascore,movie,votes,year
0,8.1,77,Logan,483078,(2017)
1,7.5,76,Wonder Woman,412999,(2017)
2,8.0,94,Dunkirk,390888,(2017)
3,7.3,85,Star Wars: Episode VIII - The Last Jedi,383070,(2017)
4,7.7,67,Guardians of the Galaxy Vol. 2,381605,(2017)
5,7.9,74,Thor: Ragnarok,335389,(2017)
6,7.5,73,Spider-Man: Homecoming,326916,(2017)
7,7.7,84,Get Out,306545,(I) (2017)
8,7.7,86,Baby Driver,300342,(2017)
9,8.1,81,Blade Runner 2049,296172,(2017)


### Script pour pages Multiples

We'll scrape the first 4 pages of each year in the interval 2000-2017. 4 pages for each of the 18 years makes for a total of 72 pages. Each page has 50 movies, so we'll scrape data for 3600 movies at most. But not all the movies have a Metascore, so the number will be lower than that. Even so, we are still very likely to get data for over 2000 movies.

##### Voilà le topo quoi

##### Changing Url's Parameters

In [20]:
#liste des 4 pages à parcourir
pages = [str(i) for i in range(1,5)]
#liste des années à parcourir
years_url = [str(i) for i in range(2000,2018)]

##### Controlling the crawl-rate

Il est très important de controller le flux de crawl exercé sur un serveur sous peine de se faire bannir.
* Pour ce faire on utilisera '.sleep()'.
* On utilise une fonction aléatoire pour imiter le comportement humain avec 'randint()'


In [21]:
from time import sleep
from random import randint

##### Monitoring the loop as it's still going

Given that we're scraping 72 pages, it would be nice if we could find a way to monitor the scraping process as it's still going. This feature is definitely optional, but it can be very helpful in the testing and debugging process.

For our script, we'll make use of this feature, and monitor the following parameters:

   * The frequency (speed) of requests, so we make sure our program is not overloading the server.
   * The number of requests, so we can halt the loop in case the number of expected requests is exceeded.
   * The status code of our requests, so we make sure the server is sending back the proper responses.


In [22]:
from time import time
from IPython.core.display import clear_output

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.4970670311845121 requests/s


In [23]:
""" Vérifier si i ly a un problème (status code != 200)"""
from warnings import warn

warn('Warning Simulation')

c:\users\marwane\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Warning Simulation
  after removing the cwd from sys.path.


### Piecing everything together

In [24]:
from requests import get

In [25]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

In [26]:
# Preparing the monitoring of the loop
start_time = time()
requests = 0

In [31]:
for year in years_url:
    
    for page in pages:
        
        #requete GET
        response = get('http://www.imdb.com/search/title?release_date=' + year + 
        '&sort=num_votes,desc&page=' + page)
        
        #pause de loop
        sleep(randint(8,15))
        
        #Afficher la requete et monitorer
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        #renvoyer un warning pour les non 'status code : 200'
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
        #Sortir de la boucle si le nombre de requetes est plus grand qu'attendu
        if requests > 72:
            warn('Number of requests was greater than expected.')  
            break 
            
        #Parser en BS
        page_html = BeautifulSoup(response.text,'html.parser')
        
        # choisir les 50 movies d'une page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        #extraire les données
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
        
        
        
        
        

ConnectionError: HTTPConnectionPool(host='www.imdb.com', port=80): Max retries exceeded with url: /search/title?release_date=2013&sort=num_votes,desc&page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x093279D0>: Failed to establish a new connection: [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu',))

In [32]:
len(names)

3421

### Examining the scraped data

In [33]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421 entries, 0 to 3420
Data columns (total 5 columns):
imdb         3421 non-null float64
metascore    3421 non-null int64
movie        3421 non-null object
votes        3421 non-null int64
year         3421 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 106.9+ KB
None


,imdb,metascore,movie,votes,year
0,8.5,67,Gladiator,1133246,(2000)
1,8.5,80,Memento,973459,(2000)
2,8.3,55,Snatch,682753,(2000)
3,8.3,68,Requiem for a Dream,661613,(2000)
4,7.4,64,X-Men,509159,(2000)
5,7.8,73,Cast Away,446991,(2000)
6,7.6,64,American Psycho,406218,(2000)
7,7.2,62,Unbreakable,293193,(2000)
8,7.0,73,Meet the Parents,282809,(2000)
9,6.1,59,Mission: Impossible II,268756,(2000)


In [34]:
movie_ratings.tail()

,imdb,metascore,movie,votes,year
3416,5.6,27,Coyote Ugly,97544,(2000)
3417,7.2,56,Men of Honor,97252,(2000)
3418,8.1,85,Faa yeung nin wa,96432,(2000)
3419,6.7,42,The Family Man,93420,(2000)
3420,7.4,67,Frequency,90488,(2000)


In [35]:
df.to_csv('movies_imdb.csv')

In [36]:
df.isnull().sum()

imdb         0
metascore    0
movie        0
votes        0
year         0
dtype: int64

### Cleaning the scraped data

We'll clean the scraped data with two goals in mind: plotting the distribution of IMDB and Metascore ratings, and sharing the dataset. Consequently, our data cleaning will consist of:

   * Reordering the columns.
   * Cleaning the year column and convert the values to integers.
   * Checking the extreme rating values to determine if all the ratings are within the expected intervals.
   * Normalizing one of the ratings type (or both) for generating a comparative histogram.


In [37]:
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes']]
movie_ratings.head()

,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1133246
1,Memento,(2000),8.5,80,973459
2,Snatch,(2000),8.3,55,682753
3,Requiem for a Dream,(2000),8.3,68,661613
4,X-Men,(2000),7.4,64,509159


##### Convertir 'Year' en integer

In [38]:
movie_ratings['year'].unique()

array(['(2000)', '(I) (2000)', '(1999)', '(2008)', '(2009)', '(1998)',
       '(1975)', '(1992)', '(2004)', '(1994)', '(2012)', '(1981)',
       '(2015)', '(1995)', '(2016)', '(1993)', '(1980)', '(2001)',
       '(2007)', '(1997)', '(2014)', '(2005)', '(1984)', '(1988)',
       '(1979)', '(2013)', '(2006)', '(2002)', '(1971)', '(2011)',
       '(2010)', '(1983)', '(1989)', '(1976)', '(1982)', '(1966)',
       '(1987)', '(1986)', '(1996)', '(1957)', '(I) (2013)', '(I) (2008)',
       '(I) (2012)', '(1968)', '(1960)', '(I) (2011)', '(2017)',
       '(I) (2015)', '(2003)', '(I) (2004)', '(I) (2005)'], dtype=object)

In [39]:
""" récupérer les nombres en moonwalk (commencer par la fin)"""

' récupérer les nombres en moonwalk (commencer par la fin)'

In [40]:
movie_ratings.loc[:,'year'] = movie_ratings['year'].str[-5:-1].astype(int)#pchaaaaakh

In [41]:
movie_ratings['year'].head(3)

0    2000
1    2000
2    2000
Name: year, dtype: int32

In [42]:
movie_ratings.describe()

,year,imdb,metascore,votes
count,3421.000000,3421.000000,3421.000000,3.421000e+03
mean,2002.768196,7.656270,70.825782,5.034624e+05
std,10.469437,0.696805,15.158810,1.979932e+05
min,1957.000000,4.800000,24.000000,9.048800e+04
25%,2000.000000,7.200000,61.000000,4.555090e+05
50%,2003.000000,7.900000,72.000000,5.375740e+05
75%,2011.000000,8.100000,83.000000,6.223210e+05
max,2017.000000,8.900000,98.000000,1.414365e+06


In [43]:
""" Describe() sur les valeurs min et max de chaque type"""
movie_ratings.describe().loc[['min','max'],['imdb','metascore']]

,imdb,metascore
min,4.8,24.0
max,8.9,98.0


In [44]:
### Normaliser les données imdb et metascore sur la même échelle (sur 100). donc il faut normaliser imdb (sui est sur 10)
movie_ratings['n_imdb'] = movie_ratings['imdb']*10
movie_ratings.head(3)

,movie,year,imdb,metascore,votes,n_imdb
0,Gladiator,2000,8.5,67,1133246,85.0
1,Memento,2000,8.5,80,973459,85.0
2,Snatch,2000,8.3,55,682753,83.0


##### Sauvegarder le tableau

In [47]:
movie_ratings.to_csv('movie_ratings.csv',encoding='utf-8')